In [1]:
import numpy as np
import pandas as pd
import pickle
import jieba
import json
import re

In [2]:
train_data_path = '../data/atec_nlp_sim_train.csv'  # 训练数据
train_add_data_path = '../data/atec_nlp_sim_train_add.csv'  # 添加训练数据
stop_words_path = '../data/stop_words.txt'  # 停用词路径
tokenize_dict_path = '../data/dict_all.txt'  # jieba分词新自定义字典
spelling_corrections_path = '../data/spelling_corrections.json'

In [3]:
train_data_df = pd.read_csv(train_data_path, sep='\t', header=None,names=["index", "s1", "s2", "label"])
train_add_data_df = pd.read_csv(train_add_data_path, sep='\t', header=None, names=["index", "s1", "s2", "label"])
train_all = pd.concat([train_data_df, train_add_data_df])

In [4]:
train_all = train_all.sample(frac=1).reset_index(drop=True)  

In [5]:
train_all.head()

,index,s1,s2,label
0,42417,花呗退款后，还款金额没变化,我的还款金额与花呗显示的不一样,0
1,22304,为什么我的花呗不能充话费了,为什么支付到了五百以后一天就不能用花呗了,0
2,57490,淘宝分期可以现金和花呗一起,花呗分期和花呗一起结算,0
3,39613,花呗分期付款怎么还钱,花呗怎么能提额,0
4,29605,蚂蚁借呗怎么样还款,网商银行里的蚂蚁借呗怎么还款,1


### 分词及处理

In [6]:
jieba.load_userdict(tokenize_dict_path)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/bj/34ww9tjd5mz5wrq2jqgqrtw40000gn/T/jieba.cache
Loading model cost 0.796 seconds.
Prefix dict has been built succesfully.


In [7]:
# 停用词表
stopwords = [line.strip() for line in open(stop_words_path, 'r', encoding='utf-8').readlines()]

In [8]:
# 拼错词替换表
with open(spelling_corrections_path,"r",encoding="utf-8") as file:
    spelling_corrections = json.load(file)

In [9]:
def transform_other_word(str_text,reg_dict):
    """
    替换词
    :param str_text:待替换的句子
    :param reg_dict:替换词字典
    :return:
    """
    for token_str,replac_str in reg_dict.items():
        str_text = str_text.replace(token_str, replac_str)
    return str_text

In [10]:
def seg_sentence(sentence, stop_words):
    """
    对句子进行分词
    :param sentence:句子，停用词
    """
    sentence_seged = jieba.cut(sentence.strip())
    word_list = [i for i in sentence_seged if i not in stop_words and i != ' ']
    return word_list

In [11]:
def preprocessing_word(s1_train, s2_train, stopwords, spelling_corrections):

    # 去除句子中的脱敏数字***，替换成一
    re_object = re.compile(r'\*+')

    s1_all = []
    s2_all = []
    all = []

    for s1_,s2_ in zip(s1_train, s2_train):
        s1 = re_object.sub(u"十一", s1_)
        s2 = re_object.sub(u"十一", s2_)
        spell_corr_s1 = transform_other_word(s1, spelling_corrections)
        spell_corr_s2 = transform_other_word(s2, spelling_corrections)

        # 分词
        seg_s1 = seg_sentence(spell_corr_s1, stopwords)
        seg_s2 = seg_sentence(spell_corr_s2, stopwords)

        all.extend(seg_s1)
        all.extend(seg_s2)
        s1_all.append(seg_s1)
        s2_all.append(seg_s2)
    source_list = []
    # source_list = list(set(all))
    source_list.append('<UNK>')
    source_list.append('<PAD>')
    source_list.extend(list(set(all)))
    word2id = {}
    id2word = {}
    for index, char in enumerate(source_list):
        word2id[char] = index
        id2word[index] = char

    return s1_all, s2_all, word2id, id2word

In [12]:
s1_train = train_all["s1"].tolist()
s2_train = train_all["s2"].tolist()
y_train = train_all["label"].tolist()

In [13]:
s1_word_all, s2_word_all, word2id, id2word = preprocessing_word(s1_train, s2_train, stopwords, spelling_corrections)

In [14]:
def make_word2id(data, word2id):
    data2id = []
    for word_list in data:
        id_list = [word2id.get(i) if word2id.get(i) is not None else word2id.get('<PAD>') for i in word_list]
        data2id.append(id_list)
    return data2id

In [15]:
def all_data_set(s1_all, s2_all, word2id, y_train, max_l=15):
    pad = word2id['<PAD>']
    all_data = []
    s1_data_id = make_word2id(s1_all, word2id)
    s2_data_id = make_word2id(s2_all, word2id)
    s1_all_new = []
    s2_all_new = []
    y = []
    for i in range(len(s1_data_id)):
        if len(s1_data_id[i]) > max_l:
            s1_set = s1_data_id[i][:max_l]
        else:
            s1_set = np.concatenate((s1_data_id[i], np.tile(pad, max_l - len(s1_data_id[i]))), axis=0)
        if len(s2_data_id[i]) > max_l:
            s2_set = s2_data_id[i][:max_l]
        else:
            s2_set = np.concatenate((s2_data_id[i], np.tile(pad, max_l - len(s2_data_id[i]))), axis=0)
        y_set = [1,0] if y_train[i] == 0 else [0,1]
        s1_all_new.append(s1_set)
        s2_all_new.append(s2_set)
        y.append(y_set)
    return s1_all_new, s2_all_new, y

In [16]:
s1_word_id_all, s2_word_id_all, y_set = all_data_set(s1_word_all, s2_word_all, word2id, y_train, max_l=15)

In [17]:
train_all["s1_word_all"] = s1_word_all

In [18]:
train_all["s2_word_all"] = s2_word_all

In [19]:
train_all["s1_word_id_all"] = s1_word_id_all

In [20]:
train_all["s2_word_id_all"] = s2_word_id_all

In [21]:
train_all["y_set"] = y_set

In [22]:
train_all.tail()

,index,s1,s2,label,s1_word_all,s2_word_all,s1_word_id_all,s2_word_id_all,y_set
102472,35011,用花呗满减吗,花呗付款不是说减***元吗,0,"[用花呗, 满减]","[花呗, 付款, 不是, 说, 减, 十一元]","[3647, 4264, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[4448, 12431, 10963, 1859, 8171, 11944, 1, 1, ...","[1, 0]"
102473,6850,花呗一天在同一家店能用多少次,花呗一个月能使用多少笔,0,"[花呗, 一天, 同, 一家, 店, 能用, 多少]","[花呗, 一个月, 能, 使用, 多少, 笔]","[4448, 8612, 6157, 4942, 11269, 3863, 5127, 1,...","[4448, 10288, 10649, 10582, 5127, 5246, 1, 1, ...","[1, 0]"
102474,39911,可你们不能因为这个把我的花呗给关闭了吧,麻烦你帮忙看看我的花呗，现在用不了了,0,"[可, 你们, 不能, 因为, 把, 花呗, 给关]","[麻烦, 帮忙, 看看, 花呗, 现在, 用不了]","[1899, 10110, 2609, 4072, 3082, 4448, 4137, 1,...","[7490, 12199, 171, 4448, 8403, 1497, 1, 1, 1, ...","[1, 0]"
102475,60262,更换支付宝账号。花呗怎么办,支付宝账户的花呗怎么更换,0,"[更换, 支付宝, 账号, 花呗, 怎么办]","[支付宝, 账号, 花呗, 怎么, 更换]","[2107, 5007, 5135, 4448, 5619, 1, 1, 1, 1, 1, ...","[5007, 5135, 4448, 3871, 2107, 1, 1, 1, 1, 1, ...","[1, 0]"
102476,9834,花呗每周二权益日,花呗每周二立减***元吗,0,"[花呗, 每周, 二, 权益日]","[花呗, 每周, 二立, 减, 十一元]","[4448, 3259, 808, 675, 1, 1, 1, 1, 1, 1, 1, 1,...","[4448, 3259, 9087, 8171, 11944, 1, 1, 1, 1, 1,...","[1, 0]"


In [23]:
# 将数据存到一个大列表里面，格式是[[s1,s2,y],[s1,s2,y],[s1,s2,y].......]
all_data = []
for i in range(len(s1_word_id_all)):
    all_data.append([s1_word_id_all[i],s2_word_id_all[i],y_set[i]])

In [24]:
ratio = int(len(all_data)*0.8)
train_data = all_data[:ratio]
test_data = all_data[ratio:]

In [25]:
# 将数据存入pickle中
with open("word_data.pk", 'wb') as f1:
    pickle.dump((train_data,test_data,word2id,id2word), f1)